In [ ]:
!kaggle competitions download -c playground-series-s5e3
!unzip playground-series-s5e3

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
  0%|                                               | 0.00/59.0k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 59.0k/59.0k [00:00<00:00, 1.46MB/s]
Archive:  playground-series-s5e3.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!rm -rf playground-series-s5e3.zip

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Competition

Based on ROC, illustrates the performance of a Binary Classifier.

From the statement, we know that this must be able to guess the probability. The first things that comes to my mind are:

- KNN
- Trees
- Random Forest
- SVMs
- Logistic Regression
- XGBoost

## Data Analysis

In [ ]:
df = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.duplicated().sum()

In [ ]:
df.isnull().sum()

In [ ]:
df.nunique()

- **Rainfall**: although is a binary 0-1, we can see `mean` is not near 0.5. Then we have to take into account this for our model
- **Reescaling is necessary**, as this is pure numerical value data.
- `winddirection, humidity, cloud` have kinda unique values. This might led to categorical variables

In [ ]:
target_variable = "rainfall"
numerical_variables = list(filter(lambda e: e != target_variable, df.columns))

### Numerical Analysis

In [ ]:
# Function to create and display a row of plots for a single variable
def create_variable_plots(df, variable):
    sns.set_style('whitegrid')
    fig, axes = plt.subplots(1, 2, figsize=(10, 4))

    # Box plot
    sns.boxplot(x=df[variable], ax=axes[0])
    axes[0].set_xlabel(variable)
    axes[0].set_title(f"Box Plot for {variable}")

    # Histogram
    sns.histplot(df[variable], kde=True, bins=30, ax=axes[1])
    axes[1].set_xlabel(variable)
    axes[1].set_ylabel("Frequency")
    axes[1].set_title(f"Histogram for {variable}")

    # Adjust spacing between subplots
    plt.tight_layout()
    plt.show()

def create_variable_pie_plot(df, variable):
    sns.set_style('whitegrid')
    counts = df[variable].value_counts()
    
    fig, ax = plt.subplots(figsize=(10, 4))
    
    ax.pie(counts, labels=counts.index, autopct='%1.1f%%', startangle=90, 
           colors=sns.color_palette("pastel"), wedgeprops={'edgecolor': 'black'})
    
    plt.title(f"Pie chart of {variable}")
    plt.tight_layout()
    plt.show()

In [ ]:
for num_var in numerical_variables:
    create_variable_plots(df, num_var)

create_variable_pie_plot(df, target_variable)

Outliers are considered when the value is 1.5 bigger than the IQR (Q3 - Q1). We can see that: `pressure`, `mintemp` (just 1 outlier), `dewpoint`, `humidity`, `cloud` and `windspeed` have several outliers.

A more extensive analysis from each variable:

`pressure`: follows *almost* a Normal Distribution at around 1013 (from the boxplot) -> need to smooth values 

`maxtemp`: left skewed two peaks. Seasons(? -> binning (agrupar) into temperature ranges to capture seasonality.

`temperature`: left skewed two peaks. Seasons(? -> need to do a relation with max_temp and min_temp

`min_temp`: left sweked two peaks (much bigger the right one) -> as there are some outliers, we can try to smooth data

`dewpoint`: left sweked. Lots of outliers -> log transformation for reducing outliers

`humidity`: outliers. left skewed (most data >= 75%) -> what happens with low humidity?

`cloud`: outliers. left skewed (most data >= 70%) -> correlation with humidity

`sunshine`: highly right skewed. Near 0 -> see negative correlation with cloud and humidity

`winddirection`: bimodal distribution. 50º and 200º. -> hot encoding for capturing possible patterns

`windspeed`: right skewed. Some outliers -> log transformation


Seems that `rainfall` is unbalanced, which will probably affect to the model performance.

- Since the data is skewed toward rainy days, models may struggle to correctly predict non-rainy days.
- We might need resampling techniques (e.g., SMOTE, undersampling) or adjusted class weights to improve balance.


### Bivariate Analysis

In [ ]:
# Before
plt.figure(figsize=(10, 7))
sns.heatmap(df.select_dtypes(include=["int64", "float64"]).corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.show()

In [ ]:
# after, more easy to understand, as repeteated variables are not displayed
variables = [col for col in df.columns if col in numerical_variables]

# Adding variables to the existing list
train_variables = variables 

# Calculate correlation matrices for train_data and test_data
corr_train = df[train_variables].corr()


# Create masks for the upper triangle
mask_train = np.triu(np.ones_like(corr_train, dtype=bool))

# Set the text size and rotation
annot_kws = {"size": 8, "rotation": 45}

# Generate heatmaps for train_data
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
ax_train = sns.heatmap(corr_train, mask=mask_train, cmap='viridis', annot=True,
                      square=True, linewidths=.5, xticklabels=1, yticklabels=1, annot_kws=annot_kws)
plt.title('Correlation Heatmap - Train Data')

# Adjust layout
plt.tight_layout()

# Show the plots
plt.show()

We should also do a Pairplot usedo for discerning the `kernel density estimation`, also known as KDE, helps us to “smooth” and explore data that doesn't follow any typical probability density distribution, such as normal distribution, binomial distribution, etc.

Most of our data don't follow a normal distr. then, its a good idea to do this.

In [ ]:
# Generate pairplot for train_data
plt.figure(figsize=(10, 6))
sns.pairplot(df.drop(columns=['id'], errors='ignore'), diag_kind="kde", corner=True)
plt.suptitle("Pairplot - Train Data", fontsize=40, y=1.05)
plt.show()

What do we get from this? Tons of things:

1. From the first graphics, we can see very high positive correlations (>0.8). This means that some data can be dropped and still not lose a lot of information. This happens specially with `maxTemp`, `pressure`, `minTemp` and `dewPoint` -> drop some data (feature selection)
2. From the first graphics, we can see high correlations (0.5 around). `cloud`--`humidity`. `winddirection` -- `maxTemp`, `pressure`, `minTemp` and `dewPoint`. -> scale or transform data to improve the model
3. From the first graphics, we can see high negative correlations. `cloud` -- `sunshine`. -> drop some data (feature selection)

For correlations that are not so clear, more analysis should be considered.

## Data Preprocessing
### Feature Engineering

Remove noise and useless atributes, so it won't affect the model :)

- Enhances patterns & relationships → Helps models capture hidden trends.
- Removes noise & redundancy → Keeps only what truly matters.
- Improves accuracy & efficiency → Optimized data leads to better predictions.
- Reduces dimensionality → Fewer features = Faster computation
- 
After creating new atributes, the idea is to compute an statistical analysis again and see if we can see new relations or not.

In [ ]:
df_test.isnull().sum()

In [ ]:
df_test['winddirection'] = df_test['winddirection'].fillna(df_test['winddirection'].median())

In [ ]:
def perform_feature_engineering(df):
    # Temperature Difference - Difference between max and min temperature
    df['Temp_Diff'] = df['maxtemp'] - df['mintemp']

    # Dew Point Spread - Difference between temperature and dew point
    df['Dew_Point_Spread'] = df['temparature'] - df['dewpoint']

    # Humidity Category - Binning humidity into low, medium, and high
    df['Humidity_Category'] = pd.cut(df['humidity'], bins=[0, 50, 80, 100], labels=['Low', 'Medium', 'High'])

    # Cloud Cover Category - Grouping cloud cover into bins
    df['Cloud_Cover_Category'] = pd.cut(df['cloud'], bins=[0, 30, 70, 100], labels=['Clear', 'Partly Cloudy', 'Overcast'])

    # Sunshine Duration Category - Categorizing sunshine duration
    df['Sunshine_Category'] = pd.cut(df['sunshine'], bins=[-1, 3, 7, 13], labels=['Low', 'Medium', 'High'])

    # Wind Speed Intensity - Categorizing wind speeds
    df['Wind_Speed_Intensity'] = pd.cut(df['windspeed'], bins=[0, 10, 25, 60], labels=['Calm', 'Breezy', 'Windy'])

    # Wind Direction Grouping - Binning wind direction into 4 quadrants
    df['Wind_Quadrant'] = pd.cut(df['winddirection'], bins=[0, 90, 180, 270, 360], labels=['NE', 'SE', 'SW', 'NW'], include_lowest=True)

    # Interaction Feature: Pressure & Humidity - Multiply to capture pressure-humidity effects
    df['Pressure_Humidity_Interaction'] = df['pressure'] * df['humidity']

    # Interaction Feature: Wind & Cloud Cover - Wind effect on cloud cover
    df['Wind_Cloud_Interaction'] = df['windspeed'] * df['cloud']

    # Temperature Ratio - Normalized temperature based on max recorded value - the closer to 1.0, the hotter!
    df['Temp_Ratio'] = df['temparature'] / df['maxtemp'].max()

    return df

# Apply feature engineering to both train and test data
df = perform_feature_engineering(df)
df_test = perform_feature_engineering(df_test)

In [ ]:
df.head()

In [ ]:
df_test.head()

### Remove new outliers

But new outliers could have appeared in the new columns... Why should we remove it? **The add noise to the data, reducing the precission of the model**.

Obviously, we know which columns have to go through this part because of the previous graphs.

In [ ]:
columns_to_check = ['dewpoint', 'humidity', 'cloud', 'windspeed', 'Temp_Diff', 'Dew_Point_Spread', 'Pressure_Humidity_Interaction', 'Wind_Cloud_Interaction', 'Temp_Ratio']

# Function to remove outliers using IQR and visualize
def remove_outliers_iqr_with_plot(data, column):
    Q1 = data[column].quantile(0.15)
    Q3 = data[column].quantile(0.85)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Filter the data
    filtered_data = data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]
    
    # Calculate the number of rows deleted
    rows_deleted = len(data) - len(filtered_data)
    
    # Plot the distribution with outliers
    plt.figure(figsize=(8, 4))
    sns.boxplot(x=data[column], color='lightblue', flierprops={'marker': 'o', 'markersize': 5, 'markerfacecolor': 'red'})
    
    # Highlight Q1 and Q3
    plt.axvline(Q1, color='green', linestyle='--', label='Q1 (10th Percentile)')
    plt.axvline(Q3, color='blue', linestyle='--', label='Q3 (90th Percentile)')
    
    # Highlight lower and upper bounds
    plt.axvline(lower_bound, color='red', linestyle='-', label='Lower Bound')
    plt.axvline(upper_bound, color='red', linestyle='-', label='Upper Bound')

    plt.title(f'Outlier Detection for {column}')
    plt.legend()
    plt.xlabel(column)
    plt.show()
    
    return filtered_data, rows_deleted

# Apply function to each numerical column and visualize
rows_deleted_total = 0

for column in columns_to_check:
    df, rows_deleted = remove_outliers_iqr_with_plot(df, column)
    rows_deleted_total += rows_deleted
    print(f"Rows deleted for {column}: {rows_deleted}")

print(f"Total rows deleted: {rows_deleted_total}")

In [ ]:
df.isnull().sum()

### Transformation of distributions

Almost all real data is skewed (does not follow a normal distr.). Thus, we must try to aproximate it to one. This can be use doing transformations.

The skew-umbral depends (here it was 0.7). For ML, it might be interesting to use one that is lower.

**THIS IS USUALLY DONE FOR TRAINING TEST**

In [ ]:
# [FOR TRAIN]
# Identify features with skewness greater than 0.75
skewed_features = df[numerical_variables].skew()[df[numerical_variables].skew() > 0.70].index.values

# Print the list of variables to be transformed
print("Features to be transformed (skewness > 0.75):")
display(skewed_features)

# Plot skewed features before transformation
for feature in skewed_features:
    plt.figure(figsize=(8, 4))
    sns.histplot(df[feature], bins=50, kde=True, color='blue')
    plt.title(f'Distribution of {feature} before log transformation')
    plt.show()

# Apply log1p (log transform) transformation to skewed features
df[skewed_features] = np.log1p(df[skewed_features])

# Plot skewed features after transformation
for feature in skewed_features:
    plt.figure(figsize=(8, 4))
    sns.histplot(df[feature], bins=50, kde=True, color='green')
    plt.title(f'Distribution of {feature} after log transformation')
    plt.show()

### Feature Encoding

As always KNN (for instance), can't process categorical variables. We must hot-encode, dummy-encode or whatever in order to be able to process them.

In [ ]:
category_columns = df.select_dtypes(include=['category']).columns
for column in category_columns:
    print(column)

In [ ]:
df_encoding = df[category_columns]
df_test_encoding = df_test[category_columns]

# Dropping selected columns for scaling - the ones that are not categorical
# (numerical) must be scaled for better model performance
df_to_scale = df.drop(category_columns, axis=1)
df_test_to_scale = df_test.drop(category_columns, axis=1)

df_encoded = pd.get_dummies(df_encoding, columns=category_columns, drop_first=True)
df_test_encoded = pd.get_dummies(df_test_encoding, columns=category_columns, drop_first=True)

In [ ]:
df_encoded.sample(1)

In [ ]:
Y = df[target_variable]

### Feature Scaling

There are tons of scalers, the most common ones are:
- MinMax()
- Estandarization
- Robust Scaler()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# this could have be done with a Pipeline() if we wanted to try several scalers

# Initialize MinMaxScaler
minmax_scaler = MinMaxScaler()

# Fit the scaler on the training data
minmax_scaler.fit(df_to_scale.drop(['rainfall'], axis=1))

# Scale the training data
scaled_data_train = minmax_scaler.transform(df_to_scale.drop(['rainfall'], axis=1))
scaled_train_df = pd.DataFrame(scaled_data_train, columns=df_to_scale.drop(['rainfall'], axis=1).columns)

# Scale the test data using the parameters from the training data
scaled_data_test = minmax_scaler.transform(df_test_to_scale)
scaled_test_df = pd.DataFrame(scaled_data_test, columns=df_test_to_scale.columns)

In [ ]:
scaled_train_df.sample(3)

## Model

Wopsie, model time. After data processing, I would like to try the following models:
- KNN
- Trees
- Forest
- XgBoost - want to try this
- SVMs
- Logistic Regression
- Genetic Programming - want to try this

So we will create a model for each and see what we obtain.

### KNN Model with Cross-validation

Data split will follow my beloved 60-20-20... Wait. We can't, as we already have the Train and Test dataset.
But we can do is adecuate the train dataset for having a training and validating subset, then testing them with the test dataset.

In [ ]:
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score

In [ ]:
random_seed = 42

In [ ]:
kv = KFold(n_splits=5, shuffle=True, random_state=random_seed)
best_acc = None
best_model = None
scores = []

for neigh in range(2, 20, 2):
    model = KNeighborsClassifier(n_neighbors=neigh, weights="distance") 
    fold_scores = []  # Almacenar los scores de los 5 folds

    for train_idx, val_idx in kv.split(scaled_data_train):
        X_train, X_val = scaled_train_df.iloc[train_idx], scaled_train_df.iloc[val_idx]
        y_train, y_val = Y.iloc[train_idx], Y.iloc[val_idx]
        
        # Entrenar modelo
        model.fit(X_train, y_train)
        
        # Predecir en validación
        y_pred = model.predict(X_val)
        
        # Evaluar el modelo
        score = roc_auc_score(y_val, y_pred)
        fold_scores.append(score)  # Guardamos el score de cada fold
    
    mean_score = np.mean(fold_scores)  # Promediamos los 5 folds

    if best_acc is None or mean_score > best_acc:
        best_acc = mean_score
        best_model = model
        
    scores.append(mean_score)  # Guardamos solo el promedio de los folds

print(f"Cross-validation scores: {scores}")
print(len(scores))  # Ahora será 10 (uno por cada valor de `neigh`)
print(f"Mean CV score: {np.mean(scores)}")

In [ ]:
print(f"Best model is {best_model} with an accuracy of {best_acc}")

This precissions was done cross-eval method. However, we must execute it for the official data to test and upload it to see the accuracy.

In [ ]:
y_pred = best_model.predict(scaled_test_df)

In [ ]:
y_pred.shape

In [ ]:
scaled_test_df.shape

In [ ]:
submission = pd.DataFrame({
    "id": df_test["id"],
    "rainfall":y_pred 
})

In [ ]:
submission.head()

In [ ]:
submission.to_csv("submission.csv", index=False)

After updating it to Kaggle, we got a score of **0.68812**.

### Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
best_model = None
best_acc = 0.0
crit = ["gini", "entropy"]

for max_d in range(2, 20, 1): # max_depth = aprox log(n_rows)
    for min_s_s in range(2, 50, 1):
        for c in crit:
            model = DecisionTreeClassifier(criterion=c, max_depth=max_d, min_samples_leaf=min_s_s, random_state=random_seed)
            scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')
            score = scores.mean() # average of all cross-val

            if score > best_acc:
                best_acc = score
                best_model = model

print(f"Best model: {best_model} with precission {best_acc}")

In [ ]:
best_model.fit(X_train, y_train)
y_pred = best_model.predict(scaled_test_df)
y_pred.shape

submission = pd.DataFrame({
    "id": df_test["id"],
    "rainfall": y_pred
})

submission.to_csv("submission.csv", index=False)

In [ ]:
submission.isnull().sum()

Those were tons of iterations. But we did have what it was promissed: the model! With a Kaggle precission of: **0.77648**

### RandomForestClassifier